In [2]:
#  источники - https://www.kaggle.com/code/pierremegret/gensim-word2vec-tutorial/notebook
# https://spacy.io/models/ru#ru_core_news_md

import re  # Для препроцессинга
!pip install pandas
import pandas as pd  # Для обработки данных
from time import time  # Для определения времени операций
import spacy
from spacy.lang.ru.examples import sentences 
from collections import defaultdict  # Для частоты слов
import logging  # Настройка для gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [3]:
# Считываю файл
data = pd.read_csv('Сlean.csv')
df=data['description']
df.shape

(100723,)

In [4]:
df.head()

0    Прекрасная однокомнатная квартира в доме с лиф...
1    Светлая, просторная комната 19 кв. метров. Отд...
2    Первомайская, 15 мин. пешкомТёплая уютная квар...
3    в шаговой доступности парк Сокольники. , Разви...
4    Ильинское с.Все коммуникации центральные.Кварт...
Name: description, dtype: object

In [5]:
df.isnull().sum()

0

In [10]:
# Загружаю модель
!python -m spacy download ru_core_news_sm

     ---------------------------------------- 15.3/15.3 MB 8.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [11]:
nlp = spacy.load('ru_core_news_sm', disable=['ner', 'parser']) # отключение распознавания именованных объектов для повышения скорости
def cleaning(doc):
    #Лемматизируется и удаляет стоп-слова
    # doc должен быть пространственным объектом Doc
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec использует контекстные слова для изучения векторного представления целевого слова,
    # если предложение состоит всего из одного или двух слов,
    # польза от тренинга мала, поэтому
    if len(txt) > 2:
        return ' '.join(txt)

INFO - 03:05:14: Loading dictionaries from c:\Users\Daniel\anaconda3\lib\site-packages\pymorphy2_dicts_ru\data
INFO - 03:05:14: format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [12]:
#Удаляю неалфавитные символы:
brief_cleaning = (re.sub("[^А-Яа-я]", ' ', str(row)).lower() for row in data['description'])

In [9]:
# Атрибут space .pipe() для ускорения процесса очистки:
# Подбираю оптимальное batch_size=2000, n_process=4
t = time()
txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=2000, n_process=4)]
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 26.03 mins


In [10]:
#Помещаю результаты в фрейм данных, чтобы удалить недостающие значения и дубликаты:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(65113, 1)

In [11]:
df_clean.head(20)

,clean
0,прекрасный однокомнатный квартира дом лифт мус...
1,светлый просторный комната кв метр о...
2,первомайский мина пешкомт плая уютный к...
3,шаговый доступность парк сокольники развит...
4,ильинский коммуникация центральный квартира хо...
5,продаваться срочно однокомнатный квартира ремо...
6,знамя октябрь пос продаваться светлый уютный...
7,маяковская мина пешкомбаррикадная ...
8,продаваться двухкомнатный квартира общий площа...
9,домодедово успейте купить выгодно ...


In [ ]:
df_clean.to_csv('PRE.csv',index=True)

In [7]:
df_clean = pd.read_csv('E:\project university\Выпускная работа\PRE.csv')

In [8]:
# использую пакет фраз Gensim для автоматического определения общих фраз (биграмм) из списка предложений.
from gensim.models.phrases import Phrases, Phraser

In [9]:
# Phrases() принимает список слов в качестве входных данных:
sent = [row.split() for row in df_clean['clean']]

In [10]:
# фразы из списка предложений:
phrases = Phrases(sent, min_count=30, progress_per=10000)

INFO - 00:00:10: collecting all words and their counts
INFO - 00:00:10: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 00:00:12: PROGRESS: at sentence #10000, processed 623358 words and 173825 word types
INFO - 00:00:12: PROGRESS: at sentence #20000, processed 1187655 words and 273413 word types
INFO - 00:00:13: PROGRESS: at sentence #30000, processed 1733814 words and 343709 word types
INFO - 00:00:15: PROGRESS: at sentence #40000, processed 2866897 words and 443755 word types
INFO - 00:00:17: PROGRESS: at sentence #50000, processed 3884567 words and 528139 word types
INFO - 00:00:19: PROGRESS: at sentence #60000, processed 4895880 words and 598383 word types
INFO - 00:00:19: collected 635089 token types (unigram + bigrams) from a corpus of 5313725 words and 65113 sentences
INFO - 00:00:19: merged Phrases<635089 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 00:00:19: Phrases lifecycle event {'msg': 'built Phrases<635089 vocab, min_count=30, 

In [57]:
print(phrases)


Phrases<635089 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>


In [11]:
# Phrase() - сокращает потребление памяти Phrases(), отбросив состояние модели, которое строго не требуется для задачи обнаружения биграмм:
bigram = Phraser(phrases)

INFO - 00:00:19: exporting phrases from Phrases<635089 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 00:00:21: FrozenPhrases lifecycle event {'msg': 'exported FrozenPhrases<3862 phrases, min_count=30, threshold=10.0> from Phrases<635089 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000> in 1.99s', 'datetime': '2022-11-19T00:00:21.844445', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}


In [12]:
#Преобразование корпуса на основе обнаруженных биграмм:
sentences = bigram[sent]

In [13]:
#проверка эффективности лемматизации, удаления стоп-слов и добавления биграмм.
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

46478

In [14]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['квартира',
 'дом',
 'кв',
 'этаж',
 'двор',
 'район',
 'школа',
 'комплекс',
 'жилой',
 'кухня']

In [15]:
#использую реализацию Gensim word2vec
import multiprocessing
from gensim.models import Word2Vec

In [16]:
cores = multiprocessing.cpu_count() # Количество ядер на компе

In [17]:
#Параметры модели
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     vector_size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

INFO - 00:00:41: Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=300, alpha=0.03)', 'datetime': '2022-11-19T00:00:41.822829', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}


In [18]:
# Word2Vec требует создания таблицы словарного запаса 
# (просто переварив все слова и отфильтровав уникальные слова, а также выполнив некоторые базовые подсчеты по ним):
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 00:00:43: collecting all words and their counts
INFO - 00:00:43: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 00:00:44: PROGRESS: at sentence #10000, processed 519503 words, keeping 17344 word types
INFO - 00:00:45: PROGRESS: at sentence #20000, processed 990655 words, keeping 23260 word types
INFO - 00:00:46: PROGRESS: at sentence #30000, processed 1449491 words, keeping 27016 word types
INFO - 00:00:47: PROGRESS: at sentence #40000, processed 2315237 words, keeping 34080 word types
INFO - 00:00:48: PROGRESS: at sentence #50000, processed 3117085 words, keeping 39644 word types
INFO - 00:00:50: PROGRESS: at sentence #60000, processed 3902562 words, keeping 44006 word types
INFO - 00:00:50: collected 46478 word types from a corpus of 4245081 raw words and 65113 sentences
INFO - 00:00:50: Creating a fresh vocabulary
INFO - 00:00:50: Word2Vec lifecycle event {'msg': 'effective_min_count=20 retains 9666 unique words (20.796936184861654%% of original 4647

Time to build vocab: 0.12 mins


In [19]:
#Обучение модели
t = time()
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=10, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 00:00:54: Word2Vec lifecycle event {'msg': 'training model with 11 workers on 9666 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2 shrink_windows=True', 'datetime': '2022-11-19T00:00:54.107455', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'train'}
INFO - 00:00:55: EPOCH 1 - PROGRESS: at 8.25% examples, 116756 words/s, in_qsize 18, out_qsize 0
INFO - 00:00:56: EPOCH 1 - PROGRESS: at 23.35% examples, 152571 words/s, in_qsize 21, out_qsize 0
INFO - 00:00:57: EPOCH 1 - PROGRESS: at 40.19% examples, 172504 words/s, in_qsize 13, out_qsize 8
INFO - 00:00:58: EPOCH 1 - PROGRESS: at 51.51% examples, 182064 words/s, in_qsize 21, out_qsize 0
INFO - 00:00:59: EPOCH 1 - PROGRESS: at 60.42% examples, 188216 words/s, in_qsize 21, out_qsize 0
INFO - 00:01:00: EPOCH 1 - PROGRESS: at 69.54% examples, 193857 words/s, in_qsize 21, out_qsize 0
INFO - 00:01:01: EPOCH 1

Time to train the model: 1.51 mins


In [20]:
#Поскольку не планирую дальше обучать модель, вызываю init_sims(), что сделает модель намного более эффективной с точки зрения использования памяти:
w2v_model.init_sims(replace=True)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_10716\3670000816.py:2: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)
WARNING - 00:02:24: destructive init_sims(replace=True) deprecated & no longer required for space-efficiency


In [21]:
#Наиболее похожие слова на:
w2v_model.wv.most_similar(positive=["добираться"])

[('быстро_добраться', 0.7362722754478455),
 ('легко_добраться', 0.7165964841842651),
 ('быстро_добираться', 0.7125027775764465),
 ('добраться_точка', 0.7012991309165955),
 ('добраться', 0.683266282081604),
 ('доезжать', 0.6684644222259521),
 ('выехать', 0.6568681001663208),
 ('доехать', 0.6504923701286316),
 ('доберетесь', 0.6495130658149719),
 ('оказаться', 0.6255602240562439)]

In [22]:
#Сходство слов
w2v_model.wv.similarity("дом", 'год',)

0.20000324

In [23]:
#Лишнее слово
w2v_model.wv.doesnt_match(['паркет', 'плитка', 'люстра'])

'люстра'

In [24]:
#Значимость слов 
w2v_model.wv.most_similar(positive=['квартира', 'этаж'], negative=['транспорт'], topn=3)

[('этажность_этаж', 0.5079990029335022),
 ('кв', 0.4985583424568176),
 ('звезда_арбат', 0.4972324073314667)]

In [25]:
w2v_model.wv.index_to_key

['квартира',
 'дом',
 'кв',
 'этаж',
 'двор',
 'район',
 'школа',
 'комплекс',
 'жилой',
 'кухня',
 'москва',
 'ремонт',
 'метро',
 'два',
 'детский_сад',
 'парк',
 'жк',
 'комната',
 'детский',
 'окно',
 'отделка',
 'новый',
 'просторный',
 'находиться',
 'год',
 'минута',
 'продаваться',
 'корпус',
 'хороший',
 'сделка',
 'шаговый_доступность',
 'территория',
 'станция_метро',
 'инфраструктура',
 'центр',
 'магазин',
 'студия',
 'площадь',
 'санузел',
 'комнатный',
 'квартал',
 'продажа',
 'площадка',
 'свободный_продажа',
 'взрослый_собственник',
 'общий_площадь',
 'отличный',
 'минута_пешком',
 'собственник',
 'зона',
 'современный',
 'больший',
 'место',
 'ипотека',
 'планировка',
 'поликлиника',
 'пеший_доступность',
 'уютный',
 'вид',
 'лоджия',
 'светлый',
 'подъезд',
 'подземный_паркинг',
 'спальня',
 'расположен',
 'отдых',
 'балкон',
 'собственность',
 'гостиная',
 'окно_выходить',
 'развитой_инфраструктура',
 'высота_потолок',
 'сторона',
 'бизнес_класс',
 'сделать',
 'боль